# Setup

In [1]:
from collections import Counter
import copy
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


import statsmodels.formula.api as smf
import xgboost as xgb
from sklearn.calibration import IsotonicRegression

In [2]:
from ml_func import analyze_accuracy, get_predictions, plot_calibration
from ml_func import setup_chess_data

In [3]:
X_train, X_test, y_train, y_test = setup_chess_data()

# Train and Predict

In [10]:
test_obs = list(X_train.iloc[0, :].values)

In [54]:
params = {'booster': 'gbtree', 'eval_metric': 'logloss', 'max_depth': 3,
          'gamma': 0.1, 'colsample_bytree': 1, 'subsample': 1, 'min_child_weight': 3, 'n_jobs': -1,
         'objective': 'binary:logistic'}
xgb_matrix = xgb.DMatrix(X_train, label=y_train)
booster = xgb.train(params=params, dtrain=xgb_matrix, num_boost_round=100)

In [56]:
temp = xgb.DMatrix(np.array([test_obs]), feature_names = X_train.columns)
test = booster.predict(temp)
print(test)

[0.99814034]


In [ ]:
# Testing Isotonic Fit

In [67]:
isotonic = IsotonicRegression(out_of_bounds='clip')

preds = booster.predict(xgb.DMatrix(X_train))

isotonic.fit(preds, y_train)
isotonic_probs = isotonic.predict(preds)
isotonic_probs.sort()
for x in isotonic_probs:
    print(x)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.18644068
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.2962963
0.

0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563
0.99945563

0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379
0.9995379


0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121


0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121
0.9996121


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
